In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210320.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,17/03/2021 10:22:47,Beauty is Embarrassing (2012),Bury the Hatchet (2010),"Harlan County, U.S.A. (1976)",Rodents of Unusual Size (2017),Beyond the Mat (1999),American Movie (1999),[Default to Nicolas Cage],IT'S IN BLACK AND WHITE,ARE WE THE BADDIES?,IT'S A CONSPIRACY,1982,THE VERY NEAR FUTURE,VIDEO GAMES
1,17/03/2021 10:30:10,Beyond the Mat (1999),Rodents of Unusual Size (2017),Beauty is Embarrassing (2012),Bury the Hatchet (2010),NaN,NaN,IT'S A CONSPIRACY,IT'S IN BLACK AND WHITE,1982,THE VERY NEAR FUTURE,ARE WE THE BADDIES?,[Default to Nicolas Cage],VIDEO GAMES
2,17/03/2021 10:42:03,American Movie (1999),"Harlan County, U.S.A. (1976)",Bury the Hatchet (2010),Rodents of Unusual Size (2017),Beyond the Mat (1999),Beauty is Embarrassing (2012),1982,ARE WE THE BADDIES?,IT'S A CONSPIRACY,THE VERY NEAR FUTURE,IT'S IN BLACK AND WHITE,[Default to Nicolas Cage],VIDEO GAMES
3,17/03/2021 10:49:18,"Harlan County, U.S.A. (1976)",American Movie (1999),Rodents of Unusual Size (2017),Beyond the Mat (1999),Beauty is Embarrassing (2012),Bury the Hatchet (2010),[Default to Nicolas Cage],VIDEO GAMES,ARE WE THE BADDIES?,THE VERY NEAR FUTURE,IT'S A CONSPIRACY,1982,IT'S IN BLACK AND WHITE
4,17/03/2021 10:55:10,"Harlan County, U.S.A. (1976)",American Movie (1999),Bury the Hatchet (2010),Rodents of Unusual Size (2017),Beyond the Mat (1999),Beauty is Embarrassing (2012),ARE WE THE BADDIES?,IT'S A CONSPIRACY,VIDEO GAMES,IT'S IN BLACK AND WHITE,THE VERY NEAR FUTURE,1982,[Default to Nicolas Cage]
5,17/03/2021 14:52:20,Bury the Hatchet (2010),Rodents of Unusual Size (2017),"Harlan County, U.S.A. (1976)",Beauty is Embarrassing (2012),NaN,NaN,1982,THE VERY NEAR FUTURE,ARE WE THE BADDIES?,IT'S A CONSPIRACY,VIDEO GAMES,[Default to Nicolas Cage],IT'S IN BLACK AND WHITE
6,17/03/2021 14:55:01,Bury the Hatchet (2010),American Movie (1999),Beauty is Embarrassing (2012),Rodents of Unusual Size (2017),"Harlan County, U.S.A. (1976)",Beyond the Mat (1999),ARE WE THE BADDIES?,1982,IT'S A CONSPIRACY,THE VERY NEAR FUTURE,IT'S IN BLACK AND WHITE,VIDEO GAMES,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Beauty is Embarrassing (2012)', 'Bury the Hatchet (2010)',
       'Harlan County, U.S.A. (1976)', 'Rodents of Unusual Size (2017)',
       'Beyond the Mat (1999)', 'American Movie (1999)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Beauty is Embarrassing (2012)': <Candidate('Beauty is Embarrassing (2012)')>,
 'Bury the Hatchet (2010)': <Candidate('Bury the Hatchet (2010)')>,
 'Harlan County, U.S.A. (1976)': <Candidate('Harlan County, U.S.A. (1976)')>,
 'Rodents of Unusual Size (2017)': <Candidate('Rodents of Unusual Size (2017)')>,
 'Beyond the Mat (1999)': <Candidate('Beyond the Mat (1999)')>,
 'American Movie (1999)': <Candidate('American Movie (1999)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Beauty is Embarrassing (2012), Bury the Hatchet (2010), Harlan County, U.S.A. (1976), Rodents of Unusual Size (2017), Beyond the Mat (1999), American Movie (1999))>,
 <Ballot(Beyond the Mat (1999), Rodents of Unusual Size (2017), Beauty is Embarrassing (2012), Bury the Hatchet (2010))>,
 <Ballot(American Movie (1999), Harlan County, U.S.A. (1976), Bury the Hatchet (2010), Rodents of Unusual Size (2017), Beyond the Mat (1999), Beauty is Embarrassing (2012))>,
 <Ballot(Harlan County, U.S.A. (1976), American Movie (1999), Rodents of Unusual Size (2017), Beyond the Mat (1999), Beauty is Embarrassing (2012), Bury the Hatchet (2010))>,
 <Ballot(Harlan County, U.S.A. (1976), American Movie (1999), Bury the Hatchet (2010), Rodents of Unusual Size (2017), Beyond the Mat (1999), Beauty is Embarrassing (2012))>,
 <Ballot(Bury the Hatchet (2010), Rodents of Unusual Size (2017), Harlan County, U.S.A. (1976), Beauty is Embarrassing (2012))>,
 <Ballot(Bury the Hatchet (2010), American Movie 

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                         Votes  Status
------------------------------  -------  --------
Bury the Hatchet (2010)               2  Hopeful
Harlan County, U.S.A. (1976)          2  Hopeful
American Movie (1999)                 1  Hopeful
Beauty is Embarrassing (2012)         1  Hopeful
Beyond the Mat (1999)                 1  Rejected
Rodents of Unusual Size (2017)        0  Rejected

ROUND 2
Candidate                         Votes  Status
------------------------------  -------  --------
Bury the Hatchet (2010)               2  Hopeful
Harlan County, U.S.A. (1976)          2  Hopeful
Beauty is Embarrassing (2012)         2  Hopeful
American Movie (1999)                 1  Rejected
Beyond the Mat (1999)                 0  Rejected
Rodents of Unusual Size (2017)        0  Rejected

ROUND 3
Candidate                         Votes  Status
------------------------------  -------  --------
Harlan County, U.S.A. (1976)          3  Hopeful
Bury the Hatchet (2010)               

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
ARE WE THE BADDIES?              2  Hopeful
1982                             2  Hopeful
[Default to Nicolas Cage]        2  Hopeful
IT'S A CONSPIRACY                1  Rejected
IT'S IN BLACK AND WHITE          0  Rejected
VIDEO GAMES                      0  Rejected
THE VERY NEAR FUTURE             0  Rejected

ROUND 2
Candidate                    Votes  Status
-------------------------  -------  --------
1982                             3  Hopeful
ARE WE THE BADDIES?              2  Hopeful
[Default to Nicolas Cage]        2  Rejected
IT'S A CONSPIRACY                0  Rejected
IT'S IN BLACK AND WHITE          0  Rejected
VIDEO GAMES                      0  Rejected
THE VERY NEAR FUTURE             0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
ARE WE THE BADDIES?              4  Elected
1982                             3  